# 도구 호출 검토 방법 (기능적 API)

!!! 정보 "사전 요구 사항"
    이 가이드는 다음에 대한 이해를 전제로 합니다:

    - [인간-순환](../../concepts/human_in_the_loop) 워크플로우 구현 [인터럽트](../../concepts/human_in_the_loop/#interrupt) 사용
    - [기능적 API를 사용하여 ReAct 에이전트 생성 방법](../../how-tos/react-agent-from-scratch-functional)

이 가이드는 LangGraph [기능적 API](../../concepts/functional_api)를 사용하여 ReAct 에이전트에서 인간-순환 워크플로우를 구현하는 방법을 보여줍니다.

우리는 [기능적 API를 사용하여 ReAct 에이전트를 생성하는 방법](../../how-tos/react-agent-from-scratch-functional) 가이드에서 생성된 에이전트를 기반으로 작업할 것입니다.

특히, 우리는 [채팅 모델](https://python.langchain.com/docs/concepts/chat_models/)에 의해 생성된 [도구 호출](https://python.langchain.com/docs/concepts/tool_calling/)을 실행하기 전에 검토하는 방법을 보여줄 것입니다. 이는 애플리케이션의 주요 포인트에서 [인터럽트](../../concepts/human_in_the_loop/#interrupt) 기능을 사용하여 수행할 수 있습니다.

**미리 보기**:

우리는 채팅 모델에서 생성된 도구 호출을 검토하는 간단한 함수를 구현하고 이를 애플리케이션의 [엔트리포인트](../../concepts/functional_api/#entrypoint) 내부에서 호출할 것입니다:

```python
def review_tool_call(tool_call: ToolCall) -> Union[ToolCall, ToolMessage]:
    """도구 호출을 검토하고 검증된 버전을 반환합니다."""
    human_review = interrupt(
        {
            "question": "이것이 올바른가요?",
            "tool_call": tool_call,
        }
    )
    review_action = human_review["action"]
    review_data = human_review.get("data")
    if review_action == "continue":
        return tool_call
    elif review_action == "update":
        updated_tool_call = {**tool_call, **{"args": review_data}}
        return updated_tool_call
    elif review_action == "feedback":
        return ToolMessage(
            content=review_data, name=tool_call["name"], tool_call_id=tool_call["id"]
        )
```

## 설정

먼저, 필요한 패키지를 설치하고 API 키를 설정합시다:


In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


<div class="admonition tip">
     <p class="admonition-title">더 나은 디버깅을 위해 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
     <p style="padding-top: 5px;">
         LangSmith에 가입하여 문제를 빠르게 발견하고 LangGraph 프로젝트의 성능을 개선하세요. LangSmith는 LangGraph로 만든 LLM 애플리케이션을 디버깅, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있게 해줍니다 — 시작하는 방법에 대해서는 <a href="https://docs.smith.langchain.com">문서</a>를 읽어보세요.
     </p>
 </div>


## 모델 및 도구 정의

먼저 예제를 위해 사용할 도구와 모델을 정의합시다. [ReAct 에이전트 가이드](../../how-tos/react-agent-from-scratch-functional)와 같이, 우리는 위치에 대한 날씨 설명을 얻는 단일 플레이스홀더 도구를 사용할 것입니다.

이번 예제에서는 [OpenAI](https://python.langchain.com/docs/integrations/providers/openai/) 채팅 모델을 사용할 것이지만, 도구 호출을 지원하는 어떤 모델도 [사용할 수](https://python.langchain.com/docs/integrations/chat/) 있습니다.


In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

model = ChatOpenAI(model="gpt-4o-mini")


@tool
def get_weather(location: str):
    """Call to get the weather from a specific location."""
    # This is a placeholder for the actual implementation
    if any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "It's sunny!"
    elif "boston" in location.lower():
        return "It's rainy!"
    else:
        return f"I am not sure what the weather is in {location}"


tools = [get_weather]


## 작업 정의

우리의 [작업](../../concepts/functional_api/#task)은 [ReAct 에이전트 가이드](../../how-tos/react-agent-from-scratch-functional)와 변경되지 않았습니다:

1. **모델 호출**: 우리는 메시지 목록으로 우리의 채팅 모델에 쿼리하고자 합니다.
2. **도구 호출**: 만약 우리의 모델이 도구 호출을 생성한다면, 우리는 이를 실행하고자 합니다.


In [2]:
from langchain_core.messages import ToolCall, ToolMessage
from langgraph.func import entrypoint, task


tools_by_name = {tool.name: tool for tool in tools}


@task
def call_model(messages):
    """Call model with a sequence of messages."""
    response = model.bind_tools(tools).invoke(messages)
    return response


@task
def call_tool(tool_call):
    tool = tools_by_name[tool_call["name"]]
    observation = tool.invoke(tool_call["args"])
    return ToolMessage(content=observation, tool_call_id=tool_call["id"])


## 엔트리포인트 정의

실행 전에 도구 호출을 검토하기 위해, 우리는 `review_tool_call` 함수를 추가하여 [interrupt](../../concepts/human_in_the_loop/#interrupt)를 호출합니다. 이 함수가 호출되면, 재개 명령을 내릴 때까지 실행이 일시 중지됩니다.

주어진 도구 호출에 대해, 우리의 함수는 인간 검토를 위해 `interrupt`를 수행합니다. 그 시점에서 우리는 다음을 수행할 수 있습니다:

- 도구 호출을 수락합니다;
- 도구 호출을 수정하고 계속 진행합니다;
- 사용자 지정 도구 메시지를 생성합니다 (예: 모델에게 도구 호출 형식을 다시 지정하도록 지시하는 경우).

아래의 [사용 예시](#usage)에서 이 세 가지 사례를 보여줄 것입니다.


In [3]:
from typing import Union


def review_tool_call(tool_call: ToolCall) -> Union[ToolCall, ToolMessage]:
    """Review a tool call, returning a validated version."""
    human_review = interrupt(
        {
            "question": "Is this correct?",
            "tool_call": tool_call,
        }
    )
    review_action = human_review["action"]
    review_data = human_review.get("data")
    if review_action == "continue":
        return tool_call
    elif review_action == "update":
        updated_tool_call = {**tool_call, **{"args": review_data}}
        return updated_tool_call
    elif review_action == "feedback":
        return ToolMessage(
            content=review_data, name=tool_call["name"], tool_call_id=tool_call["id"]
        )


이제 생성된 도구 호출을 검토하기 위해 [진입점](../../concepts/functional_api/#entrypoint)을 업데이트할 수 있습니다. 도구 호출이 수락되거나 수정되면 이전과 동일한 방식으로 실행합니다. 그렇지 않으면 인간이 제공한 `ToolMessage`를 그냥 추가합니다.

!!! 팁

    이전 작업의 결과 — 이 경우 초기 모델 호출 — 는 지속되어 `interrupt` 이후에 다시 실행되지 않습니다.


In [4]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langgraph.types import Command, interrupt


checkpointer = MemorySaver()


@entrypoint(checkpointer=checkpointer)
def agent(messages, previous):
    if previous is not None:
        messages = add_messages(previous, messages)

    llm_response = call_model(messages).result()
    while True:
        if not llm_response.tool_calls:
            break

        # Review tool calls
        tool_results = []
        tool_calls = []
        for i, tool_call in enumerate(llm_response.tool_calls):
            review = review_tool_call(tool_call)
            if isinstance(review, ToolMessage):
                tool_results.append(review)
            else:  # is a validated tool call
                tool_calls.append(review)
                if review != tool_call:
                    llm_response.tool_calls[i] = review  # update message

        # Execute remaining tool calls
        tool_result_futures = [call_tool(tool_call) for tool_call in tool_calls]
        remaining_tool_results = [fut.result() for fut in tool_result_futures]

        # Append to message list
        messages = add_messages(
            messages,
            [llm_response, *tool_results, *remaining_tool_results],
        )

        # Call model again
        llm_response = call_model(messages).result()

    # Generate final response
    messages = add_messages(messages, llm_response)
    return entrypoint.final(value=llm_response, save=messages)


### 사용 예시

몇 가지 시나리오를 보여드리겠습니다.


In [5]:
def _print_step(step: dict) -> None:
    for task_name, result in step.items():
        if task_name == "agent":
            continue  # just stream from tasks
        print(f"\n{task_name}:")
        if task_name in ("__interrupt__", "review_tool_call"):
            print(result)
        else:
            result.pretty_print()


### 도구 호출 수락하기

도구 호출을 수락하기 위해, 우리가 `Command`에서 제공하는 데이터에 도구 호출이 통과해야 한다고 표시하기만 하면 됩니다.


In [6]:
config = {"configurable": {"thread_id": "1"}}


In [7]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

for step in agent.stream([user_message], config):
    _print_step(step)


{'role': 'user', 'content': "What's the weather in san francisco?"}

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_Bh5cSwMqCpCxTjx7AjdrQTPd)
 Call ID: call_Bh5cSwMqCpCxTjx7AjdrQTPd
  Args:
    location: San Francisco

__interrupt__:
(Interrupt(value={'question': 'Is this correct?', 'tool_call': {'name': 'get_weather', 'args': {'location': 'San Francisco'}, 'id': 'call_Bh5cSwMqCpCxTjx7AjdrQTPd', 'type': 'tool_call'}}, resumable=True, ns=['agent:22fcc9cd-3573-b39b-eea7-272a025903e2'], when='during'),)


In [8]:
# highlight-next-line
human_input = Command(resume={"action": "continue"})

for step in agent.stream(human_input, config):
    _print_step(step)



call_tool:
================================= Tool Message =================================

It's sunny!

call_model:
================================== Ai Message ==================================

The weather in San Francisco is sunny!


### 도구 호출 수정

도구 호출을 수정하려면 업데이트된 인수를 제공할 수 있습니다.


In [9]:
config = {"configurable": {"thread_id": "2"}}


In [10]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

for step in agent.stream([user_message], config):
    _print_step(step)


{'role': 'user', 'content': "What's the weather in san francisco?"}

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_b9h8e18FqH0IQm3NMoeYKz6N)
 Call ID: call_b9h8e18FqH0IQm3NMoeYKz6N
  Args:
    location: san francisco

__interrupt__:
(Interrupt(value={'question': 'Is this correct?', 'tool_call': {'name': 'get_weather', 'args': {'location': 'san francisco'}, 'id': 'call_b9h8e18FqH0IQm3NMoeYKz6N', 'type': 'tool_call'}}, resumable=True, ns=['agent:9559a81d-5720-dc19-a457-457bac7bdd83'], when='during'),)


In [11]:
# highlight-next-line
human_input = Command(resume={"action": "update", "data": {"location": "SF, CA"}})

for step in agent.stream(human_input, config):
    _print_step(step)



call_tool:
================================= Tool Message =================================

It's sunny!

call_model:
================================== Ai Message ==================================

The weather in San Francisco is sunny!


이번 실행의 LangSmith 추적은 특히 유익합니다:

- [중단 전](https://smith.langchain.com/public/c8b07579-5cf4-4adb-a849-282163bc9d99/r/b5b128d6-e715-480b-b58d-59e64f724275) 추적에서, 우리는 위치 `"샌프란시스코"`에 대한 도구 호출을 생성합니다.
- [재개 후](https://smith.langchain.com/public/b28b92e5-a555-482d-aa4d-c675a19f0eb5/r) 추적에서는 메시지의 도구 호출이 `"SF, CA"`로 업데이트된 것을 확인할 수 있습니다.


### 사용자 지정 ToolMessage 생성

사용자 지정 `ToolMessage`를 생성하기 위해, 메시지의 내용을 제공합니다. 이 경우 모델에 도구 호출의 형식을 다시 조정하도록 요청할 것입니다.


In [12]:
config = {"configurable": {"thread_id": "3"}}


In [13]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

for step in agent.stream([user_message], config):
    _print_step(step)


{'role': 'user', 'content': "What's the weather in san francisco?"}

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_VqGjKE7uu8HdWs9XuY1kMV18)
 Call ID: call_VqGjKE7uu8HdWs9XuY1kMV18
  Args:
    location: San Francisco

__interrupt__:
(Interrupt(value={'question': 'Is this correct?', 'tool_call': {'name': 'get_weather', 'args': {'location': 'San Francisco'}, 'id': 'call_VqGjKE7uu8HdWs9XuY1kMV18', 'type': 'tool_call'}}, resumable=True, ns=['agent:4b3b372b-9da3-70be-5c68-3d9317346070'], when='during'),)


In [14]:
# highlight-next-line
human_input = Command(
    # highlight-next-line
    resume={
        # highlight-next-line
        "action": "feedback",
        # highlight-next-line
        "data": "Please format as <City>, <State>.",
        # highlight-next-line
    },
    # highlight-next-line
)

for step in agent.stream(human_input, config):
    _print_step(step)



call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_xoXkK8Cz0zIpvWs78qnXpvYp)
 Call ID: call_xoXkK8Cz0zIpvWs78qnXpvYp
  Args:
    location: San Francisco, CA

__interrupt__:
(Interrupt(value={'question': 'Is this correct?', 'tool_call': {'name': 'get_weather', 'args': {'location': 'San Francisco, CA'}, 'id': 'call_xoXkK8Cz0zIpvWs78qnXpvYp', 'type': 'tool_call'}}, resumable=True, ns=['agent:4b3b372b-9da3-70be-5c68-3d9317346070'], when='during'),)


한 번 재형식화되면, 우리는 그것을 받아들일 수 있습니다:


In [15]:
# highlight-next-line
human_input = Command(resume={"action": "continue"})

for step in agent.stream(human_input, config):
    _print_step(step)



call_tool:
================================= Tool Message =================================

It's sunny!

call_model:
================================== Ai Message ==================================

The weather in San Francisco, CA is sunny!
